In [1]:
import requests
import urllib
import json
import operator
import pandas as pd
from requests_html import HTML
from requests_html import HTMLSession

In [2]:
def get_url(url):
    try:
        session = HTMLSession()
        respose = session.get(url)
        return respose
    except requests.exceptions.RequestException as e:
        print(e)

In [3]:
def get_result(query):
    response = urllib.parse.quote_plus(query)#helps in building a query Url 
    response = get_url("https://suggestqueries.google.com/complete/search?output=chrome&hl=en&q=" + query)
    results = json.loads(response.text)
    return results


In [4]:
#Like quote(), but also replace spaces with plus signs,
#as required for quoting HTML form values when building up a query string to go into a URL. Plus signs in the original string are escaped unless they are included in safe.
#It also does not have safe default to '/'.

In [5]:
def order_result(result):
    suggested  = []
    for index,value in enumerate(result[1]) : 
        x = {"term":value,"relevance": result[4]['google:suggestrelevance'][index]}
        suggested.append(x)
    return suggested
        
        
        
        

In [6]:
def query_expansion(query):
    expanded_term_suffixes = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
                             'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

    expanded_term_prefixes = ['who is *', 'what is *', 'where is *', 'when can *', 'why is *', 
                              'how to *', 'best', 'cheap', 'worst', 'is', 'what', 'when', 'why', 
                              'how', 'who']
    term = []
    
    for val in expanded_term_prefixes:
        y = val + ' ' + query 
        term.append(y)
    
    
    for val in expanded_term_suffixes:
        y = query + ' ' + val
        term.append(y)
    return term 

In [7]:
def wrapped_with_expansion(query):
    all_results = []

    expanded_terms = query_expansion(query)
    for term in expanded_terms:
        result = get_result(term)
        result = order_result(result)
        all_results = all_results + result
        all_results = sorted(all_results, key=lambda k: k['relevance'], reverse=True)
    return all_results
    

In [8]:
def wrapped_without_exapnsion(query):
    result = get_result(query)
    result = order_result(result)
    return result 

In [9]:
wrapped_without_exapnsion("amazon")
    

[{'term': 'amazon india', 'relevance': 950},
 {'term': 'https://www.amazon.in/', 'relevance': 949},
 {'term': 'amazon prime', 'relevance': 700},
 {'term': 'amazon prime video', 'relevance': 601},
 {'term': 'amazon sale', 'relevance': 600},
 {'term': 'amazon customer care number', 'relevance': 552},
 {'term': 'amazon customer care number mumbai', 'relevance': 551},
 {'term': 'amazon customer care number kalyan', 'relevance': 550}]

In [10]:
wrapped_with_expansion("amazon")

[{'term': 'amazon music', 'relevance': 1252},
 {'term': 'amazon usa', 'relevance': 1252},
 {'term': 'best amazon prime series', 'relevance': 1251},
 {'term': 'cheap amazon products', 'relevance': 1251},
 {'term': 'amazon mobile', 'relevance': 1251},
 {'term': 'amazon prime', 'relevance': 1251},
 {'term': 'amazon quiz', 'relevance': 1251},
 {'term': 'amazon tracking', 'relevance': 1251},
 {'term': 'amazon uk', 'relevance': 1251},
 {'term': 'amazon youth offer', 'relevance': 1251},
 {'term': 'best amazon prime movies', 'relevance': 1250},
 {'term': 'cheap amazon prime', 'relevance': 1250},
 {'term': 'worst amazon products', 'relevance': 1250},
 {'term': 'when amazon sale will start', 'relevance': 1250},
 {'term': 'amazon app', 'relevance': 1250},
 {'term': 'amazon business', 'relevance': 1250},
 {'term': 'amazon customer care number', 'relevance': 1250},
 {'term': 'amazon fresh', 'relevance': 1250},
 {'term': 'amazon great indian festival', 'relevance': 1250},
 {'term': 'amazon india', '